In [1]:
# PLSA
import time
from zipfile import ZipFile
file_name = "test_data/2022-ntust-information-retrieval-hw4.zip"

with ZipFile(file_name, 'r',) as zip:
    zip.extractall(path = 'test_data/extractHere/2022-ntust-information-retrieval-hw4')
    print('Done!!')
    All_lst = zip.namelist()

start = time.time()

# 建dataset的所有檔名列表
Query_lst = []
Doc_lst = []

for i in All_lst:
    if "documents/" in i:
        Doc_lst.append(i)
    elif "queries/" in i:
        Query_lst.append(i)

print("Query size: ", len(Query_lst),' / ',"Document size: ", len(Doc_lst))

end = time.time()
print("execution time: %f" %(end - start))

Done!!
Query size:  100  /  Document size:  10000
execution time: 0.001000


In [2]:
# 可以在這建立lexicon
lexicon = {}
data_addr = 'test_data/extractHere/2022-ntust-information-retrieval-hw4/'

start = time.time()
for i in Query_lst:
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        lexicon[term] = 0

# 要把document所有字也加進lexicon?
# BG_L = 0
# for i in Doc_lst:
#     file = open(data_addr + i, errors = 'ignore').read().split()
#     for term in file:
# #         lexicon[term] = 0
#         BG_L = BG_L + 1

print("number of terms: ", len(lexicon))

end = time.time()
print("execution time(all small n): %f" %(end - start))

number of terms:  347
execution time(all small n): 0.220958


In [3]:
import numpy as np

start = time.time()
count_in_Q = []
for i in Query_lst:
    count = lexicon.copy()
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        if term in lexicon:
            count[term] = count[term] + 1
    
    count_in_Q.append(list(count.values()))

count_in_Q = np.array(count_in_Q)

count_in_D = []
for i in Doc_lst:
    count = lexicon.copy()
    file = open(data_addr + i, errors = 'ignore').read().split()
    for term in file:
        if term in lexicon:
            count[term] = count[term] + 1
    
    count_in_D.append(list(count.values()))

count_in_D = np.array(count_in_D)
end = time.time()
print("execution time(all small n): %f" %(end - start))

execution time(all small n): 37.318604


In [4]:
print("count_in_Q length: ", len(count_in_Q))
print("count_in_D length: ", len(count_in_D))

count_in_Q length:  100
count_in_D length:  10000


In [5]:
from numba import jit
import time

@jit()
def Likelyhood(V, D, count):
    L = 0.0
    for wi in range(V):
        for dj in range(D):
            L = L* np.power(count[dj, wi]/ (np.sum(count[dj])+1), count[dj, wi])
    return L

In [6]:
@jit()
def EM_step(curr_P_wT, curr_P_Td, prev_P_wT, prev_P_Td, P_Tkwd_tmp, count, V, D, K):
    ######
    for wi in range(V):
        for dj in range(D):
            P_Tkwd_tmp = np.multiply(prev_P_wT[wi], prev_P_Td.T[dj])
            if np.sum(P_Tkwd_tmp) == 0:
                continue
            P_Tkwd_tmp = P_Tkwd_tmp / np.sum(P_Tkwd_tmp)               
            curr_P_wT[wi] = curr_P_wT[wi] + count[dj, wi]* P_Tkwd_tmp
    sum_1 = np.sum(curr_P_wT, axis = 0)
    for Tk in range(K):
        if sum_1[Tk] == 0.0:
            continue
        curr_P_wT.T[Tk] = curr_P_wT.T[Tk]/sum_1[Tk]
    
    ######
    for dj in range(D):
        for wi in range(V):
            P_Tkwd_tmp = np.multiply(prev_P_Td.T[dj], prev_P_wT[wi])
            if np.sum(P_Tkwd_tmp) == 0:
                continue
            P_Tkwd_tmp = P_Tkwd_tmp / np.sum(P_Tkwd_tmp)
            curr_P_Td.T[dj] = curr_P_Td.T[dj] + count[dj][wi]* P_Tkwd_tmp
    
    for dj in range(D):
        curr_P_Td.T[dj] = curr_P_Td.T[dj]/np.sum(count[dj])
    
    return curr_P_wT, curr_P_Td

In [7]:
def PLSA(max_iter, curr_P_wT, curr_P_Td, P_Tkwd_tmp, count, V, D, K):
    print("Training.....")
    for iteration in range(max_iter):
        start = time.time()
        P_Tkwd_tmp = np.zeros(LT, dtype = np.float64)
        prev_P_wT = np.copy(curr_P_wT)
        prev_P_Td = np.copy(curr_P_Td)
        
#         L = Likelyhood(V, D, count)
#         print("\nLikelyhood: ", L)
    
        curr_P_wT = np.zeros([V, K], dtype = np.float64)
        curr_P_Td = np.zeros([K, D], dtype = np.float64)
        curr_P_wT, curr_P_Td = EM_step(curr_P_wT, curr_P_Td, prev_P_wT, prev_P_Td, P_Tkwd_tmp, count, V, D, K)
        
        print("Iteration: ", iteration + 1, "-------time cost: ", round(time.time()-start, 3))

In [8]:
# latent_Topics
# initialize
LT = 30
max_iter = 70
absV = len(lexicon)
absD = len(Doc_lst)

BG_L = np.sum(count_in_D)
count_in_BG = np.sum(count_in_D, axis=0)

P_Tkwd = np.zeros(LT, dtype = np.float64)
P_wT = np.random.random([absV, LT])
P_wT = P_wT / np.sum(P_wT)
P_Td = np.random.random([LT, absD])
P_Td = P_Td / np.sum(P_Td)

PLSA(max_iter, P_wT, P_Td, P_Tkwd, count_in_D, absV, absD, LT)
print("\nDone!")

Training.....

Likelyhood:  0.0
Iteration:  1 -------time cost:  5.509

Likelyhood:  0.0
Iteration:  2 -------time cost:  3.03

Likelyhood:  0.0
Iteration:  3 -------time cost:  3.025

Likelyhood:  0.0
Iteration:  4 -------time cost:  2.991

Likelyhood:  0.0
Iteration:  5 -------time cost:  2.995

Likelyhood:  0.0
Iteration:  6 -------time cost:  2.961

Likelyhood:  0.0
Iteration:  7 -------time cost:  2.875

Likelyhood:  0.0
Iteration:  8 -------time cost:  2.797

Likelyhood:  0.0
Iteration:  9 -------time cost:  2.891

Likelyhood:  0.0
Iteration:  10 -------time cost:  2.792

Likelyhood:  0.0
Iteration:  11 -------time cost:  2.756

Likelyhood:  0.0
Iteration:  12 -------time cost:  2.882

Likelyhood:  0.0
Iteration:  13 -------time cost:  2.801

Likelyhood:  0.0
Iteration:  14 -------time cost:  2.793

Likelyhood:  0.0
Iteration:  15 -------time cost:  2.796

Likelyhood:  0.0
Iteration:  16 -------time cost:  2.791

Likelyhood:  0.0
Iteration:  17 -------time cost:  2.795

Likelyhoo

In [9]:
@jit()
def SIM_CAL(D, V, count_in_Q, count_in_D, P_wT, P_Td ,ans, Alpha, Beta, count_in_BG, BG_L, sim_list):
    for dj in range(D):
        for wi in range(V):
            if count_in_Q[ans, wi] == 0:
                continue

            sum_1 = np.sum(np.multiply(P_wT[wi], P_Td.T[dj]))

            prob1 = np.log(Alpha) + np.log(count_in_D[dj, wi]/(np.sum(count_in_D[dj])+1))
            prob2 = np.log(Beta) + np.log(sum_1)
            prob3 = np.log(1-Alpha-Beta) + np.log(count_in_BG[wi]/(BG_L+1))
            prob123 = np.logaddexp(np.logaddexp(prob1, prob2), prob3)

            sim_list[dj] = sim_list[dj] + prob123
    return sim_list

In [10]:
import csv
import winsound
import time
import math

start = time.time()
# 每一迴圈輸出一個Query的結果到test.csv
with open("test_data/test.csv", "w", newline='') as Answer_csv:
    Answer_csv.truncate()
    writer = csv.writer(Answer_csv)
    writer.writerow(['Query', 'RetrievedDocuments'])
    
    for ans in range(len(Query_lst)):
        Alpha = 0.1
        Beta = 0.1
        
        sim_list = np.zeros(len(Doc_lst))
        
        sim_list = SIM_CAL(len(Doc_lst), len(lexicon), count_in_Q, count_in_D, P_wT, P_Td ,ans, Alpha, Beta, count_in_BG, BG_L, sim_list)
        
        sorted_Doc = sorted(range(len(sim_list)), key=lambda k: sim_list[k], reverse = True)

        ans_str = Doc_lst[sorted_Doc[0]].replace('documents/', '').replace(".txt", '')
        #MAP3000
        for i in range(1, 3000): 
            ans_str = ans_str + ' ' + Doc_lst[sorted_Doc[i]].replace('documents/', '').replace(".txt", '')
        
        writer.writerow([Query_lst[ans].replace('queries/', '').replace(".txt", '')] + [ans_str])

end = time.time()
print("execution time: %f" %(end - start))

duration = 500  # millisecond
freq = 440  # Hz
for i in range(5):
    winsound.Beep(freq, duration)

execution time: 3.395604
